# Section: BLAS/LAPACK - Linear Equations


Adapted from: [https://github.com/gjbex/Fortran-MOOC/tree/master/source_code/blas_lapack/linear_equations](https://github.com/gjbex/Fortran-MOOC/tree/master/source_code/blas_lapack/linear_equations)

## This program demonstrates solving linear algebra equations in Fortran.

### Linear Equation Problem

In this notebook we will use Fortran to solve a system of linear equations of the form:

$$
\Large A \mathbf{x} = \mathbf{b}
$$
where: <br>


$$
\Large A =
\left[
\begin{array}{cccc}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \vdots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{array}
\right]
$$

$$
\Large \mathbf{x} =
\left[
\begin{array}{c}
x_{1}  \\
x_{2}  \\
\vdots \\
x_{n}
\end{array}
\right]
$$

$$
\Large \mathbf{b} = 
\left[
\begin{array}{c}
b_{1}  \\
b_{2}  \\
\vdots \\
b_{n}
\end{array}
\right]
$$

The approach will be to have a main program that reads in as parameters the number of equations, the matrix $\mathbf{A}$ and the vector $\mathbf{x}$ as text files.  The main program is called *solve_equations.f90*.  It makes use of the LAPACK linear equation solver routine called **DGESV**.  Information on this routine can be found at [LAPACK - DGESV](https://netlib.org/lapack/explore-html/d7/d3b/group__double_g_esolve_ga5ee879032a8365897c3ba91e3dc8d512.html#ga5ee879032a8365897c3ba91e3dc8d512).

The main program makes use of a module called *linalg_mod.f90*, which contains subroutines to read and write matrices and arrays to and from text files.

Finally, there is a helper program called *generate_array.f90* which has a subroutine that gets arguments from the command line and generates either of vector or matrix of specified size.  Every element in the array or matrix is a random number.

The individual program files are listed below:

### In file linalg_mod.f90

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/src/linalg_mod.f90
---
language: fortran
---
```

### In file generate_array.f90

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/app/generate_array.f90
---
language: fortran
---
```

### In solve_equations.f90

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/app/solve_equations.f90
---
language: fortran
---
```

The above programs are compiled and run using Fortran Package Manager (fpm):

## Build the Program using FPM (Fortran Package Manager)

In [1]:
import os
root_dir = ""
root_dir = os.getcwd()

Since the code makes use of the LAPACK library, the following FPM configuration file (fpm.toml) was used:

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/fpm.toml
---
language: toml
---
```

In [2]:
code_dir = root_dir + "/" + "Fortran_Code/Section_BLAS_LAPACK_Linear_Equations"

In [3]:
os.chdir(code_dir)

The files *solve_equations.f90* and *generate_array.f90* were placed into the "app" folder, while the file *linalg_mod.f90* was placed into the "src" folder.

In [4]:
build_status = os.system("fpm build 2>/dev/null")

## Run the Program using FPM (Fortran Package Manager)

### Solve a Test Linear System of Two Equations

As our first run, we wish to solve the following set of linear equations:

$$
\begin{align*}
2x+8y & = 20 \\
x+2y  & = 4
\end{align*}
$$

The variables in the equations are converted into components of the $\mathbf{x}$ vector as shown below:

$$
\begin{align*}
2x_1+8x_2 & = 20 \\
x_1+2x_2  & = 4
\end{align*}
$$

These equations are converted into matrix form as shown below:

$$
\begin{equation*}
\left[
\begin{array}{cc}
2 & 8 \\
1 & 2 \\
\end{array}
\right]
\left[
\begin{array}{c}
x_1 \\
x_2 \\
\end{array}
\right]
=
\left[
\begin{array}{c}
20 \\
4 \\
\end{array}
\right]
\end{equation*}
$$

Therefore we have the following:

$$
\mathbf{A} = 
\left[
\begin{array}{cc}
2 & 8 \\
1 & 2 
\end{array}
\right]
$$

$$
\mathbf{x} = 
\left[
\begin{array}{c}
x_1 \\
x_2  
\end{array}
\right]
$$


$$
\mathbf{b} = 
\left[
\begin{array}{c}
20 \\
4  
\end{array}
\right]
$$


The matrix $\mathbf{A}$ and the vector $\mathbf{b}$ are written into text files as shown below:

In [5]:
%%writefile A_test1.txt
2 8
1 2

Overwriting A_test1.txt


In [6]:
%%writefile b_test1.txt
20
4

Overwriting b_test1.txt


The *solve_equations* program can now be run with the number of equations command line argument set to 2, and the files *A_test1.txt* and *b_test1.txt*

In [7]:
exec_status = \
    os.system("fpm run solve_equations 2>/dev/null -- 2 A_test1.txt b_test1.txt")

     -0.200000000000000E+01
      0.300000000000000E+01


The results are printed in scientfic notation and in the order of $x_1$, $x_2$.

We now wish to use Python's Numpy library to test these results:

In [8]:
import numpy as np

A = np.genfromtxt("A_test1.txt")
b = np.genfromtxt("b_test1.txt")
x = np.linalg.solve(A, b)
print("x1 = {0:2.1f}".format(x[0]))
print("x2 = {0:2.1f}".format(x[1]))

x1 = -2.0
x2 = 3.0


We can see that the Fortran code and Numpy produce the same results.

### Solve a Test Linear System of Equations of Arbitrary Size

The Fortran code can be used to solve an arbitrarily large system of equations.  To test this functionality, we make use of the *generate_array.f90* program to generate arrays or matrices of arbitrary size filled with random numbers.

As a start, we will use the *generate_array.f90* to generate a matrix file A_test2.txt that contains a 10x10 matrix.

In [9]:
exec_status = os.system("fpm run generate_array 2>/dev/null -- 10 10 > A_test2.txt") 

The $\mathbf{A}$ matrix is shown below:

In [10]:
import pandas as pd
A = pd.read_table("A_test2.txt", 
    header=None, 
    sep='\s+')
A

,0,1,2,3,4,5,6,7,8,9
0,0.479059,0.018255,0.148529,0.868123,0.652005,0.051371,0.566791,0.646060,0.430780,0.813868
1,0.948968,0.001345,0.396227,0.706450,0.240061,0.004235,0.122635,0.658306,0.481734,0.493693
2,0.487909,0.297930,0.288298,0.531290,0.805373,0.889581,0.116428,0.480135,0.919205,0.856963
3,0.008240,0.697264,0.140707,0.987016,0.136896,0.917161,0.109125,0.339215,0.881406,0.482073
4,0.803257,0.834973,0.266232,0.525780,0.696429,0.916220,0.330767,0.085754,0.044459,0.076469
5,0.745642,0.346349,0.088949,0.275074,0.530497,0.531867,0.111653,0.450347,0.184724,0.269379
6,0.093903,0.912523,0.782977,0.718646,0.366289,0.088915,0.564593,0.678411,0.871130,0.191260
7,0.564059,0.499795,0.375514,0.414760,0.232315,0.735868,0.060799,0.053368,0.101157,0.990241
8,0.383391,0.826573,0.243387,0.127267,0.148193,0.244269,0.738519,0.259511,0.649513,0.280199
9,0.724082,0.597383,0.553630,0.272111,0.611309,0.260843,0.516537,0.237010,0.728137,0.905719


And now we generate the $\mathbf{b}$ vector:

In [11]:
exec_status = os.system("fpm run generate_array 2>/dev/null -- 10 > b_test2.txt") 

The $\mathbf{b}$ vector is shown below:

In [12]:
b = pd.read_table("b_test2.txt", 
    header=None, 
    sep='\s+')
b

,0
0,0.210498
1,0.764398
2,0.051053
3,0.970386
4,0.316946
5,0.780281
6,0.968970
7,0.683564
8,0.226975
9,0.447995


We now use the *solve_equation* Fortran code to solve this linear system of equations:

In [13]:
exec_status = \
    os.system("fpm run solve_equations 2>/dev/null -- 10 A_test2.txt b_test2.txt")

      0.379509964087763E+00
      0.330127214720091E+01
     -0.181957572101974E+01
      0.582252336130065E+00
      0.355930375439009E+00
     -0.215991473028111E+01
     -0.296149549471929E+01
      0.960411217453677E+00
     -0.192534071254801E+00
      0.925350149549067E+00


And the results are compared to the output of Numpy:

In [14]:
A = np.genfromtxt("A_test2.txt")
b = np.genfromtxt("b_test2.txt")
x = np.linalg.solve(A, b)

for i in range(len(x)):
    print ("x{0:d} = {1:2.6f}".format(i+1, x[i]))

x1 = 0.379510
x2 = 3.301272
x3 = -1.819576
x4 = 0.582252
x5 = 0.355930
x6 = -2.159915
x7 = -2.961495
x8 = 0.960411
x9 = -0.192534
x10 = 0.925350


Again, we see that the results are the same.